In [1]:
# KPMG Forecasting Project
# feature selection- PCA
# Author: Taotao Jiang
# Date: June 9, 2021

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('C:/Users/jtt/Desktop/KPMG/data.csv')
df['date'] = df[['year','quarter']].apply(lambda x : '{}q{}'.format(x[0],x[1]), axis=1)
df.drop(['year','quarter'], inplace=True, axis=1, errors='ignore')
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df = df.set_index('date')

In [3]:
split = 0.3
y = df.dropna().dv
x = df.dropna().drop(['dv'], axis=1)
i = int(len(x)*(1-split))
x_train = x.iloc[:i]
y_train = y.iloc[:i]
x_test = x.iloc[i:]
y_test = y.iloc[i:]
# Z-score the features
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)
pca = PCA(n_components = 0.95)
pca.fit(x_train_scaled)
x_train_reduced = pca.transform(x_train_scaled)
x_test_reduced = pca.transform(x_test_scaled)
pca.explained_variance_ratio_

array([0.30178762, 0.20362477, 0.14622582, 0.1175854 , 0.08929466,
       0.06804988, 0.0411827 ])

In [4]:
# original regression
lr = LinearRegression().fit(x_train,y_train)
y_pred = lr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print(f'MSE = {mse}')
# pca + regression
lr_pca = LinearRegression().fit(x_train_reduced,y_train)
y_pred_pca = lr_pca.predict(x_test_reduced)
mse = mean_squared_error(y_test, y_pred_pca)
print(f'PCA_MSE = {mse}')

MSE = 0.060744038047818506
PCA_MSE = 0.060741454114100375


After PCA, 

1) the features are reduced to 7, only 1 less than the original dataset<br> 
2) the mse metric almost the same<br> 
3) PCA doesn't reflect the meaning of orignial variables

Thus, PCA is not appropriate in this case.  